In [4]:
# v tomto kode nacitame udaje o zluceninach Hetionetu a konvertujeme ich 
# do RDF grafu "https://drugbank_ontology.org"

# importujeme potrebne kniznice
import os
import json
from rdflib import Literal, URIRef, Dataset
from rdflib.namespace import RDF, FOAF, RDFS, SKOS, Namespace

# cesta k drugbank adresaru kde JSON subory 
folder_path = "/Users/simonsnopko/Desktop/Škola/bakalarska praca/DrugBank"

# definujeme namespaces pre drugbank a bioregistry
DRUGBANK = Namespace("http://go.drugbank.com/drugs/")
BIOREGISTRY = Namespace("https://bioregistry.io/drugbank:")

# do zoznamu nacitame vsetky JSON subory v danom adresari
json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]

# vytvorenie zoznamu klucov, ktore nas zaujimaju v JSON suboroch
specific_keys = ["drugbank_id", "name", "description", "synonyms"]

#  vytvorime RDF dataset a priradime mu meno
ds = Dataset()
graph_uri = URIRef("http://drugbank_ontology.org")
g = ds.graph(graph_uri)

# nabindujeme namespaces ku grafu
g.bind("drugbank", DRUGBANK)
g.bind("bioregistry", BIOREGISTRY)
g.bind("skos", SKOS)

# prechadzame cez kazdy JSON subor
for json_file in json_files:
    with open(os.path.join(folder_path, json_file)) as f:
        # nacitame obsahu JSON suboru do slovnika
        json_dict = json.load(f)
        # prechadzamme cez kluce v JSON slovniku
        for key in json_dict.keys():
            # kontrolujeme ci je kluc v zozname specific_keys
            if key in specific_keys:
                # vytvorime URI pre dany JSON subor
                json_uri = URIRef(BIOREGISTRY + json_file[:-5])
                # pridavame prislusne triplety do RDF grafu
                if key == "drugbank_id":
                    g.add((json_uri, SKOS.notation, Literal("drugbank:" + json_dict[key])))
                elif key == "name":
                    g.add((json_uri, SKOS.prefLabel, Literal(json_dict[key], lang='en')))
                elif key == "description":
                    if json_dict[key] != None:
                        g.add((json_uri, SKOS.definition, Literal(json_dict[key])))
                elif key == "synonyms":
                    for synonym in json_dict[key]:
                        if synonym["language"] == "english":
                            g.add((json_uri, SKOS.altLabel, Literal(synonym["synonym"], lang='en')))

# ulozime vytvoreny RDF dataset do suboru vo formate nquads
ds.serialize(destination="drugbank created ontology.nq", format="nquads")


<Graph identifier=N3e5722eb1aeb467795887ed982c5189e (<class 'rdflib.graph.Dataset'>)>